<a href="https://www.kaggle.com/code/felixbennett/fraud-detection-using-multilayer-perceptron?scriptVersionId=108501438" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Fraud Detection Using Multilayer Perceptron
### Define Model

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(29 , 20)
        self.fc2 = nn.Linear(20 , 10)
        self.fc3 = nn.Linear(10 , 2)
    def forward(self,x):
        #Layer 1
        x = self.fc1(x)
        x = F.relu(x)
        
        #Layer 2
        x = self.fc2(x)
        x = F.relu(x)
        
        #Layer 3
        x = self.fc3(x)
        
        return x

### Load Data

In [2]:
import pandas as pd
df = pd.read_csv('../input/fraud-detection/creditcard.csv')
df.drop('Time',axis = 1, inplace = True)
print(df.head())

         V1        V2        V3        V4        V5        V6        V7  \
0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9       V10  ...       V21       V22       V23       V24  \
0  0.098698  0.363787  0.090794  ... -0.018307  0.277838 -0.110474  0.066928   
1  0.085102 -0.255425 -0.166974  ... -0.225775 -0.638672  0.101288 -0.339846   
2  0.247676 -1.514654  0.207643  ...  0.247998  0.771679  0.909412 -0.689281   
3  0.377436 -1.387024 -0.054952  ... -0.108300  0.005274 -0.190321 -1.175575   
4 -0.270533  0.817739  0.753074  ... -0.009431  0.798278 -0.137458  0.141267   

        V25       V26       V27       V28  Amount  Class  
0  0.1285

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.3)
out = []
for t in [train,test]:
  target = np.array(t['Class'].values)
  target_t = torch.tensor(target)
  torch.reshape(target_t, (1, len(target)))
  features = np.array(t.drop('Class', axis=1).values)
  features_t = torch.tensor(features)
  tens = torch.utils.data.TensorDataset(target_t, features_t)
  out.append(tens)


# Passing to DataLoader
train_loader = torch.utils.data.DataLoader(out[0], shuffle=True, batch_size = 100)
test_loader = torch.utils.data.DataLoader(out[1], shuffle=False, batch_size = 100)
print(len(train_loader))
print(len(test_loader))

1994
855


### Define model and loss functions

In [4]:
# Instantiate model  
model = MLP()  

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005) 

### Train Model

In [5]:
from tqdm.notebook import tqdm, trange

for labels, values ,  in tqdm(train_loader):
        # Zero out the gradients
        optimizer.zero_grad()

        # Forward pass
        x = values  
        y = model(x.float())
        loss = criterion(y, labels)
        # Backward pass
        loss.backward()
        optimizer.step()

  0%|          | 0/1994 [00:00<?, ?it/s]

Testing

In [6]:
## Testing
correct = 0
total = len(test)

with torch.no_grad():
    # Iterate through test set minibatchs 
    for  labels, values in tqdm(test_loader):
        # Forward pass
        x = values
        y = model(x.float())
        
        predictions = torch.argmax(y, dim=1)
        correct += torch.sum((predictions == labels).float())
    
print('Test accuracy: {}'.format(correct/total))

  0%|          | 0/855 [00:00<?, ?it/s]

Test accuracy: 0.9993329048156738
